In [2]:
# import API token from .env

In [3]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [4]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [5]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [6]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [7]:
len(docs)

42

In [8]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

d:\ml_projects\llm_experiments\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [10]:
query = "What did the president say about the Supreme Court"
docs = db.similarity_search(query)

In [11]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And
while you’re at it, pass the Disclose Act so Americans can know who is funding our elections.

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an
Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer,
thank you for your service.

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the
United States Supreme Court.

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our
nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## QA CHAIN

In [12]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [13]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.2, "max_length":512})

In [14]:
chain = load_qa_chain(llm, chain_type="stuff")

In [15]:
query = "What did the president say about the Supreme Court"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'One of the most serious constitutional responsibilities a President has is nominating someone to serve on'

In [21]:
query = "Who is talking in this document?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'The President'

In [22]:
query = "Who created this document?"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'President Trump'

In [23]:
query = "What is the most serious constitutional responsibilities?"
docs = db.similarity_search(query)
chain.run(input_documents = docs, question=query)

'Nominating someone to serve on the United States Supreme Court'

In [24]:
query = "When is our character formed?"
docs = db.similarity_search(query)
chain.run(input_documents = docs, question = query)

'In the moment of responsibility.'

In [17]:
query = "Summarize this document?"
docs = db.similarity_search(query)
chain.run(input_documents = docs, question = query)

'The State of the Union is strong because the American people are strong.'